In [2]:
import pandas as pd
import numpy as np
from music21 import converter, midi, interval, pitch
from mido import MidiFile
import miditoolkit
import os
from os import walk
from tokenizing_functions import convert_to_note_items
#from helper_functions import get_file_and_dirnames
#from analysis_functions import analyse_data_folder
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

PATH_TRANSPOSED = "../0_data/4_preprocessed_sets"

In [3]:
def get_file_and_dirnames(p):
    f = []
    d = []
    for (dirpath, dirnames, filenames) in walk(p):
        f.extend(filenames)
        d.extend(dirnames)
        break
    return f,d

In [4]:
MIN_DURATION_DENOMINATOR = 32
DURATION_STEPS = 64
POSITION_STEPS = 16
TICKS_PER_BEAT = 1024
TICKS_PER_MIN_DURATION = TICKS_PER_BEAT*4/MIN_DURATION_DENOMINATOR
DURATION_BINS = np.arange(TICKS_PER_MIN_DURATION, (TICKS_PER_MIN_DURATION*DURATION_STEPS)+1, TICKS_PER_MIN_DURATION, dtype=int)

start_position_tokens = 37 + DURATION_STEPS
end_position_tokens = start_position_tokens + POSITION_STEPS

word2message = {0: "Bar_None"}
for i in range(1, 37):
    word2message[i] = f"Note_on_{i+59}"
for i in range(37, start_position_tokens):
    word2message[i] = f"Note_duration_{i-36}"
for i in range(start_position_tokens, end_position_tokens):
    word2message[i] = f"Position_{i-start_position_tokens+1}/{POSITION_STEPS}"

message2word = {v: k for k, v in word2message.items()}
message2word

{'Bar_None': 0,
 'Note_on_60': 1,
 'Note_on_61': 2,
 'Note_on_62': 3,
 'Note_on_63': 4,
 'Note_on_64': 5,
 'Note_on_65': 6,
 'Note_on_66': 7,
 'Note_on_67': 8,
 'Note_on_68': 9,
 'Note_on_69': 10,
 'Note_on_70': 11,
 'Note_on_71': 12,
 'Note_on_72': 13,
 'Note_on_73': 14,
 'Note_on_74': 15,
 'Note_on_75': 16,
 'Note_on_76': 17,
 'Note_on_77': 18,
 'Note_on_78': 19,
 'Note_on_79': 20,
 'Note_on_80': 21,
 'Note_on_81': 22,
 'Note_on_82': 23,
 'Note_on_83': 24,
 'Note_on_84': 25,
 'Note_on_85': 26,
 'Note_on_86': 27,
 'Note_on_87': 28,
 'Note_on_88': 29,
 'Note_on_89': 30,
 'Note_on_90': 31,
 'Note_on_91': 32,
 'Note_on_92': 33,
 'Note_on_93': 34,
 'Note_on_94': 35,
 'Note_on_95': 36,
 'Note_duration_1': 37,
 'Note_duration_2': 38,
 'Note_duration_3': 39,
 'Note_duration_4': 40,
 'Note_duration_5': 41,
 'Note_duration_6': 42,
 'Note_duration_7': 43,
 'Note_duration_8': 44,
 'Note_duration_9': 45,
 'Note_duration_10': 46,
 'Note_duration_11': 47,
 'Note_duration_12': 48,
 'Note_duration_13

In [5]:
dir = "17_POP909-Dataset-master"
file = "111.mid"
files,_ = get_file_and_dirnames(f'{PATH_TRANSPOSED}/c)_transposed_octave/{dir}')
files.sort()
files[:10]

['002.mid',
 '003.mid',
 '004.mid',
 '005.mid',
 '006.mid',
 '007.mid',
 '008.mid',
 '009.mid',
 '010.mid',
 '011.mid']

In [6]:


def quantize_items(items, ticks_per_position=1024*4/POSITION_STEPS):
    # grid
    grids = np.arange(0, items[-1]["start"]+1, ticks_per_position, dtype=int)
    # process
    for item in items:
        index = np.argmin(abs(grids - item["start"]))
        shift = grids[index] - item["start"]
        item["start"] += shift
        item["end"] += shift
    return items 

def group_items(items, max_time, ticks_per_bar=TICKS_PER_BEAT*4):
    items.sort(key=lambda x: x["start"])
    downbeats = np.arange(0, max_time+ticks_per_bar, ticks_per_bar)
    groups = []
    for db1, db2 in zip(downbeats[:-1], downbeats[1:]):
        insiders = []
        for item in items:
            if (item["start"] >= db1) and (item["start"] < db2):
                insiders.append(item)
        overall = [db1] + insiders + [db2]
        groups.append(overall)
    return groups

def item2event(groups):
    events = []
    n_downbeat = 0
    for i in range(len(groups)):
        if 'Note' not in [item["name"] for item in groups[i][1:-1]]:
            continue
        bar_st, bar_et = groups[i][0], groups[i][-1]
        n_downbeat += 1
        events.append({
            "name": "Bar",
            "time": None,
            "value": None,
            "text": n_downbeat 
        })
        for item in groups[i][1:-1]:
            # position
            flags = np.linspace(bar_st, bar_et, POSITION_STEPS, endpoint=False)
            index = np.argmin(abs(flags-item["start"]))
            events.append({
                "name": "Position",
                "time": item["start"],
                "value": f"{index+1}/{POSITION_STEPS}",
                "text": item["start"]
            })
            if item["name"] == 'Note':
                # pitch
                events.append({
                    "name": "Note_on",
                    "time": item["start"],
                    "value": item["pitch"],
                    "text": item["pitch"]
                })
                # duration
                duration = item["end"] - item["start"]
                index = np.argmin(abs(DURATION_BINS-duration))
                events.append({
                    "name": "Note_duration",
                    "time": item["start"],
                    "value": index+1,
                    "text": f"{duration}/{DURATION_BINS[index]}"
                })
    return events




path = f'{PATH_TRANSPOSED}/c)_transposed_octave/{dir}/111.mid'
note_items = convert_to_note_items(path)
quantized_items = quantize_items(note_items)
max_time = quantized_items[-1]["end"]
grouped_items = group_items(quantized_items, max_time)
events = item2event(grouped_items)
events

[{'name': 'Bar', 'time': None, 'value': None, 'text': 1},
 {'name': 'Position', 'time': 3328, 'value': '14/16', 'text': 3328},
 {'name': 'Note_on', 'time': 3328, 'value': 67, 'text': 67},
 {'name': 'Note_duration', 'time': 3328, 'value': 3, 'text': '341/384'},
 {'name': 'Position', 'time': 3840, 'value': '16/16', 'text': 3840},
 {'name': 'Note_on', 'time': 3840, 'value': 67, 'text': 67},
 {'name': 'Note_duration', 'time': 3840, 'value': 2, 'text': '256/256'},
 {'name': 'Bar', 'time': None, 'value': None, 'text': 2},
 {'name': 'Position', 'time': 4096, 'value': '1/16', 'text': 4096},
 {'name': 'Note_on', 'time': 4096, 'value': 67, 'text': 67},
 {'name': 'Note_duration', 'time': 4096, 'value': 4, 'text': '512/512'},
 {'name': 'Position', 'time': 4352, 'value': '2/16', 'text': 4352},
 {'name': 'Note_on', 'time': 4352, 'value': 71, 'text': 71},
 {'name': 'Note_duration', 'time': 4352, 'value': 5, 'text': '683/640'},
 {'name': 'Position', 'time': 5376, 'value': '6/16', 'text': 5376},
 {'nam

In [7]:
def extract_events(input_path):
    note_items = convert_to_note_items(input_path)
    quantized_items = quantize_items(note_items)
    max_time = quantized_items[-1]["end"]
    grouped_items = group_items(quantized_items, max_time)
    events = item2event(grouped_items)
    return events

events = extract_events(path)
[message2word[f"{e['name']}_{e['value']}"] for e in events][:10]

[0, 114, 8, 39, 116, 8, 38, 0, 101, 8]